---

## RIS Format

In [76]:
"""
"""
import io
import re


TAG_KEY_MAPPING = {
    'A1': 'primary_authors',  # special: Lastname, Firstname, Suffix
    'A2': 'secondary_authors',  # special: Lastname, Firstname, Suffix
    'A3': 'tertiary_authors',  # special: Lastname, Firstname, Suffix
    'A4': 'subsidiary_authors',  # special: Lastname, Firstname, Suffix
    'AB': 'abstract',
    'AD': 'author_address',
    'AN': 'accession_number',
    'AU': 'authors',  # special
    'AV': 'location_in_archives',
    'BT': 'bt',
    'C1': 'custom1',
    'C2': 'custom2',
    'C3': 'custom3',
    'C4': 'custom4',
    'C5': 'custom5',
    'C6': 'custom6',
    'C7': 'custom7',
    'C8': 'custom8',
    'CA': 'caption',
    'CN': 'call_number',
    'CP': 'cp',
    'CT': 'title_of_unpublished_ref',
    'CY': 'place_published',
    'DA': 'date',  # special: YYYY, YYYY/MM, YYYY/MM/DD/, or YYYY/MM/DD/other info
    'DB': 'name_of_database',
    'DO': 'doi',
    'DP': 'database_provider',
    'ED': 'editor',
    'EF': 'end_file',  # ignore!
    'EP': 'end_page',
    'ER': 'end_of_reference',  # special: must be empty and last tag of record
    'ET': 'edition',
    'FN': 'file_name',  # ignore!
    'ID': 'reference_id',
    'IS': 'issue_number',
    'J1': 'journal_name_user_abbr',
    'J2': 'alternate_title',
    'JA': 'journal_name_abbr',
    'JF': 'journal_name',
    'JO': 'journal_name',
    'KW': 'keywords',  # special
    'L1': 'link_to_pdf',
    'L2': 'link_to_fulltext',
    'L3': 'related_records',
    'L4': 'figure',
    'LA': 'language',
    'LB': 'label',
    'LK': 'link_to_website',
    'M1': 'number',
    'M2': 'miscellaneous_2',
    'M3': 'type_of_work',
    'N1': 'notes',
    'N2': 'abstract',
    'NV': 'number_of_volumes',
    'OP': 'original_publication',
    'PB': 'publisher',
    'PP': 'publishing_place',
    'PT': 'type_of_publication',
    'PY': 'publication_year',  # special: YYYY/MM/DD
    'RI': 'reviewed_item',
    'RN': 'research_notes',
    'RP': 'reprint_edition',  # special: 'IN FILE', 'NOT IN FILE', or 'ON REQUEST (MM/DD/YY)'
    'SE': 'section',
    'SN': 'issn',
    'SP': 'start_page',
    'ST': 'short_title',
    'T1': 'primary_title',
    'T2': 'secondary_title',  # note: journal_title, if applicable
    'T3': 'tertiary_title',
    'TA': 'translated_author',
    'TI': 'title',
    'TT': 'translated_title',
    'TY': 'type_of_reference',  # special: must be key in REFERENCE_TYPES and first tag of record
    'U1': 'user_defined_1',
    'U2': 'user_defined_2',
    'U3': 'user_defined_3',
    'U4': 'user_defined_4',
    'U5': 'user_defined_5',
    'UR': 'url',
    'VL': 'volume',
    'VR': 'version',  # ignore!
    'VO': 'published_standard_number',
    'Y1': 'primary_date',
    'Y2': 'access_date',
}

REFERENCE_TYPES_MAPPING = {
    'ABST': 'abstract',
    'ADVS': 'audiovisual material',
    'AGGR': 'aggregated database',
    'ANCIENT': 'ancient text',
    'ART': 'art work',
    'BILL': 'bill/resolution',
    'BLOG': 'blog',
    'BOOK': 'book',
    'CASE': 'case',
    'CHAP': 'book chapter',
    'CHART': 'chart',
    'CLSWK': 'classical cork',
    'COMP': 'computer program',
    'CONF': 'conference proceeding',
    'CPAPER': 'conference paper',
    'CTLG': 'catalog',
    'DATA': 'data file',
    'DBASE': 'online database',
    'DICT': 'dictionary',
    'EBOOK': 'electronic book',
    'ECHAP': 'electronic book chapter',
    'EDBOOK': 'edited book',
    'EJOUR': 'electronic article',
    'ELEC': 'web page',
    'ENCYC': 'encyclopedia',
    'EQUA': 'equation',
    'FIGURE': 'figure',
    'GEN': 'generic',
    'GOVDOC': 'government document',
    'GRANT': 'grant',
    'HEAR': 'hearing',
    'ICOMM': 'internet communication',
    'INPR': 'in press',
    'JFULL': 'journal (full)',
    'JOUR': 'journal',
    'LEGAL': 'legal rule or regulation',
    'MANSCPT': 'manuscript',
    'MAP': 'map',
    'MGZN': 'magazine article',
    'MPCT': 'motion picture',
    'MULTI': 'online multimedia',
    'MUSIC': 'music score',
    'NEWS': 'newspaper',
    'PAMP': 'pamphlet',
    'PAT': 'patent',
    'PCOMM': 'personal communication',
    'RPRT': 'report',
    'SER': 'serial publication',
    'SLIDE': 'slide',
    'SOUND': 'sound recording',
    'STAND': 'standard',
    'STAT': 'statute',
    'THES': 'thesis/dissertation',
    'UNBILL': 'unenacted bill/resolution',
    'UNPB': 'unpublished work',
    'VIDEO': 'video recording',
}

REPEATABLE_TAGS = {'A1', 'A2', 'A3', 'A4', 'AU', 'KW', 'N1'}
IGNORE_TAGS = {'FN', 'VR', 'EF'}
START_TAGS = {'TY', 'PT'}
END_TAG = 'ER'

TAG_RE = re.compile('^([A-Z][A-Z0-9])(  - | )|^(E[FR])($|  - | )')


def parse_ris_file(fname):
    with io.open(fname, mode='r') as f:

        in_record = False
        curr_tag = None
        record = {}
        records = []

        for i, line in enumerate(f):

            if not line.strip():
                continue

            tag_match = TAG_RE.match(line)
            if tag_match:
                tag = tag_match.group(1)
                
                if tag in IGNORE_TAGS:
                    continue

                elif tag == END_TAG:
                    if in_record is False:
                        raise Exception('end tag, but already out of a record?!')
                    records.append(record)
                    record = {}
                    curr_tag = tag
                    in_record = False
                    continue
                
                elif tag in START_TAGS:
                    if in_record is True:
                        raise Exception('start tag, but already in a record?!')
                    curr_tag = tag
                    in_record = True
                    value = line[tag_match.end():].strip()
                    record[TAG_KEY_MAPPING[tag]] = REFERENCE_TYPES_MAPPING.get(value, value)
                    continue
                    
                if in_record is False:
                    raise Exception('there has been a start/end tag mismatch?!')
                
                if tag in TAG_KEY_MAPPING:
                    if tag in record:
                        raise Exception('this tag is already included in record')
                    curr_tag = tag
                    value = line[tag_match.end():].strip()
                    if tag in REPEATABLE_TAGS:
                        try:
                            record[TAG_KEY_MAPPING[tag]].append(value)
                        except KeyError:
                            record[TAG_KEY_MAPPING[tag]] = [value]
                    else:
                        record[TAG_KEY_MAPPING[tag]] = value
                else:
                    print('Skipping line:', line)

            elif curr_tag in REPEATABLE_TAGS:
                try:
                    record[TAG_KEY_MAPPING[curr_tag]].append(line.strip())
                except KeyError:
                    record[TAG_KEY_MAPPING[curr_tag]] = [line.strip()]
            else:
                print('Bad line:', line)
                
    return records

In [77]:
fname = '../data/raw/citation_formats/scopus_to_ris.ris'
records = parse_ris_file(fname)

Skipping line: US Center for Disease Control and Protection, (1999) National health and nutritional examination survey (NHANES) report, , US Center for Disease Control and Protection, Atlanta; 

Skipping line: US Center for Disease Control and Protection, (2010) National health and nutritional examination survey (NHANES) report, , US Center Disease Control and Protection, Atlanta; 

Skipping line: US Environmental Protection Agency, (2014) Questions and Answers on Dispersants, , 1.usa.gov/1MzgGnc, Accessed October 19, 2015; 

Skipping line: UK Deepwater Drilling - Implications of the Gulf of Mexico Oil Spill (2011), 1. , http://www.publications.parliament.uk/pa/cm201011/cmselect/cmenergy/450/450i.pdf, Second Report of Session 2010-2011An Independent Review of the Regulatory Regime (2011), https://www.gov.uk/government/uploads/system/uploads/attachment_data/file/48252/3875-offshore-oil-gas-uk-ind-rev.pdfUR  - https://www.scopus.com/inward/record.uri?eid=2-s2.0-84940379071&partnerID=40&m

In [78]:
records[0]

{'abstract': 'Poly (styrene-divinylbenzene) (P (St-DVB)) foams with porosity as high as 98% were prepared by the method of high internal phase emulsions (HIPEs) in one-step process. The materials exhibited superhydrophobicity and excellent oleophilicity, with the water contact angle (WCA) even exceeding 150° and oil contact angle approaching 0°. The materials fabricated with different types of Fe3O4 particles had varied hierarchical pore structures. And the adsorption capacity of the monolithic foam towards chloroform was as high as 57 g/g. Importantly, the materials soaked with oil could be regenerated effectively by means of centrifugation with oil recovery rate reaching 90%. More importantly, the monolithic PolyHIPEs (polymers obtained by the polymerization of the HIPEs) were subjected to 20 adsorption-centrifugation cycles and superior reusability was demonstrated. These features achieved with PolyHIPEs made them ideal candidates for practical oil removal applications. © 2016 Elsev

In [ ]:
import io

import pandas as pd
import textacy

In [ ]:
fname = '../data/raw/all_fields_Combined Search_Results_Final.txt'
records = []
with io.open(fname, mode='rt', encoding='utf8') as f:
    record = {}
    for i, line in enumerate(f):
        if not line.strip():
            if record:
                records.append(record)
            record = {}
        else:
            try:
                field, value = line.split(':', 1)
            except ValueError:
                print(i, line)
            record[field.strip()] = value.strip()
            
        if i > 1000:
            break
            
df = pd.DataFrame(records)

In [ ]:
df = pd.read_excel('../data/raw/Combined Search_Results_Top_3.xls')
print(df.shape)
df.head(3)

In [ ]:
df[df['Title'].notnull()]['Title']